In [1]:
import pandas as pd
import os

In [2]:
def get_IO_dir(subject=None, motion_type="dynamic"):
    """
    Parameters
    ----------
    subject : string, optional
        Subject number you need to convert it's motion data to trc in "XX" string format.
        If None, user have to insert subject number manually. The default is None.
    motion_type : TYPE, optional
        DESCRIPTION. The default is "dynamic".

    Returns
    -------
    Inputs : TYPE
        DESCRIPTION.
    Outputs : TYPE
        DESCRIPTION.
    """
    
    if subject==None:
        subject = input("insert subject number: ")
        
    # Create motion Setting File
    date = subject_details[f"S{subject}"]
    if motion_type == "static":
        input_path =  f"../Data/S{subject}/{date}/Statics/" # Inputs folder
        output_path = f"../OpenSim/S{subject}/{date}/Statics/"  # Outputs folder

    elif motion_type == "dynamic":
        input_path = f"../Data/S{subject}/{date}/Dynamics/Motion_Data/"  # Inputs folder
        output_path = f"../OpenSim/S{subject}/{date}/Dynamics/Motion_Data/"  # Outputs folder
    
    if motion_type=="static":
        input_path = setting.iloc[2,1] # Inputs folder
        output_path = setting.iloc[3,1] # Outputs folder
    elif motion_type=="dynamic":
        input_path = setting.iloc[0,1] # Inputs folder
        output_path = setting.iloc[1,1] # Outputs folder
        
    print("The print will only shows files names, not full directory")
    Inputs = os.listdir(input_path)
    print("Input files: ", Inputs)
    Outputs = list(map(lambda x:f"{x}".replace('csv','trc'), Inputs))
    print("Output files: ", Outputs)
    
    # Get inputs and outputs full directories
    Inputs = list(map(lambda file: input_path+file, Inputs))
    Outputs = list(map(lambda file: output_path+file, Outputs))
    return Inputs, Outputs

### Getting the label for converting the data

In [3]:
### Motion becomes Input
def get_markers_labels(Input, Markers_Set_Name="Amged:"):
    
    Label = pd.read_csv(Input, header=2, nrows = 0).columns.values # Getting Markers labels
    Markers_Label = [] #Define a list to receive markers
    Markers_Set_Name = Markers_Set_Name

    for i in range(0, Markers_number):
        Temp = Label[2+3*i] 
        Temp_1 = Temp.replace(Markers_Set_Name, "") #Need to remove the Marker set name
        Markers_Label.append(Temp_1)
#     print("Number of labels: ",len(Markers_Label))
    return Markers_Label

### Convert the data and save to .trc file

In [4]:
def convert_data(Input, Output):
    """
    Input & Output are pathes
    """
    Markers = pd.read_csv(Input, header=5, usecols = [*range(0, 3*Markers_number+2)])
    if os.path.exists(Output): # If we already have the file, it will be removed 
        os.remove(Output)
    Markers.to_csv(Output,  sep='\t', index=False, header = False)
    num_frames = len(Markers.iloc[:,0])
    return num_frames

### trc format need special format

In [5]:
def process_trc(Output, Markers_Label, num_frames, Markers_number=39):
    New_label_Coor = '\t'
    New_label_Mar = 'Frame#\tTime'

    for i in range(0, Markers_number):
        New_label_Coor = f"{New_label_Coor}\tX{str(i+1)}\tY{str(i+1)}\tZ{str(i+1)}"
    for i in range(0,Markers_number-1):
        New_label_Mar = f"{New_label_Mar}\t{Markers_Label[i]}\t\t"
    New_label_Mar = f"{New_label_Mar}\t{Markers_Label[Markers_number-1]}\n"
    
    Contents = 'PathFileType\t4\t' + '(X,Y,Z)\t' + Output + '\n' \
            + 'DataRate\tCameraRate\tNumFrames\tNumMarkers\tUnits\tOrigDataRate\tOrigDataStartFrame\tOrigNumFrames' + '\n' \
            + f'100\t100\t{num_frames}\t{Markers_number}\tm\t100\t1\t{num_frames}\n'
    
    with open(Output, "r+") as f:
        old = f.read() # read everything in the file
        f.seek(0) # rewind
        f.write(Contents + New_label_Mar + New_label_Coor + '\n\n' + old)

In [6]:
def csv2trc(subject=None):
    if subject==None:
        subject = input("insert subject number: ")
        
    motion_types=("dynamic", "static")
    for motion_type in motion_types:
        print(f"{motion_type} Data")
        Inputs, Outputs = get_IO_dir(subject, motion_type=motion_type)
        for Input, Output in zip(Inputs, Outputs):
            Markers_Label = get_markers_labels(Input)
            num_frames = convert_data(Input, Output)
            process_trc(Output, Markers_Label, num_frames)

In [7]:
Markers_number = 39
csv2trc("02")

dynamic Data
The print will only shows files names, not full directory
Input files:  ['S02_test.csv', 'S02_train_01.csv', 'S02_train_02.csv', 'S02_val.csv']
Output files:  ['S02_test.trc', 'S02_train_01.trc', 'S02_train_02.trc', 'S02_val.trc']
static Data
The print will only shows files names, not full directory
Input files:  ['test_static.csv', 'train_static.csv', 'val_static.csv']
Output files:  ['test_static.trc', 'train_static.trc', 'val_static.trc']
